# 使用 Amazon SageMaker Studio 笔记本

此笔记本将引导您初始化和设置环境。它会向您展示如何使用 SageMaker Python SDK 中的 Amazon S3 帮助程序方法向 Amazon Simple Storage Service (Amazon S3) 存储桶上传数据和从其下载数据。

首先初始化环境。为此，请导入所需的库并获得 SageMaker Studio 使用的默认 Amazon S3 存储桶。

In [ ]:
# Install dependencies
import pandas as pd
import boto3


# SageMaker dependencies
import sagemaker
from sagemaker import get_execution_role, session
from sagemaker.s3 import S3Downloader, S3Uploader

region= boto3.Session().region_name

# This object represents the AWS Identity and Access Management (IAM) role that you are assigned
role = sagemaker.get_execution_role()
print("Role: ", role)

sm_session = session.Session(boto3.Session())
sm = boto3.Session().client("sagemaker")
sm_runtime = boto3.Session().client("sagemaker-runtime")

使用 SageMaker Python SDK S3Downloader 方法从 Amazon S3 存储桶下载数据集。有关此下载器方法的更多信息，请参阅 [SageMaker Python SDK S3Downloader 方法](https://sagemaker.readthedocs.io/en/stable/api/utility/s3.html#sagemaker.s3.S3Downloader)。

In [ ]:
bucket = ''
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print(bucket)
prefix = 'scripts/data'

S3Downloader.download(s3_uri=f"s3://{bucket}/{prefix}/iris.csv", local_path= 'data/')

以下代码将验证数据集并将其显示在网格中。

In [ ]:
import pandas as pd
import numpy as np

shape=pd.read_csv("data/iris.csv", header=None)
shape.sample(3)

将数据集分割成训练和测试数据集。使用 SageMaker Python SDK S3Uploader 方法将其上传到 Amazon S3 存储桶。有关此上传器方法的更多信息，请参阅 [SageMaker Python SDK S3Uploader 方法](https://sagemaker.readthedocs.io/en/stable/api/utility/s3.html#sagemaker.s3.S3Uploader)。

In [ ]:
train_data = shape.sample(frac=0.8,random_state=200)
test_data = shape.drop(train_data.index)

In [ ]:
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
test_file = 'test_data.csv';
test_data.to_csv(test_file, index=False, header=True)

# Return the URLs of the uploaded file, so they can be reviewed or used elsewhere
s3url = S3Uploader.upload(train_file, 's3://{}/{}'.format(bucket, prefix + "/train", "train"))
print(s3url)
s3url = S3Uploader.upload(test_file, 's3://{}/{}'.format(bucket, prefix + "/test", "test"))
print(s3url)

### 总结

恭喜！ 您已成功初始化环境，并从 Amazon S3 存储桶上传和下载了文件。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件
- 返回至实验说明